In [1]:
import numpy as np
import pandas as pd
import random
import string
from datetime import datetime, timedelta

In [2]:
data = pd.read_excel("summary_tpsd2_variables.xlsx")
adjusted_data = data.copy()

Get an impression of the data by looking at all features  
To have a quick glance on all important columns of a features output features with the same skim type toegther

In [3]:
pd.set_option('display.max_rows', 500)
data.loc[data["skim_type"].isin(["logical", "factor", "POSIXct"])].iloc[:, [0, 1, 4,5,6,7,13,14,15,16,17]]

,skim_type,skim_variable,POSIXct.min,POSIXct.max,POSIXct.median,POSIXct.n_unique,factor.ordered,factor.n_unique,factor.top_counts,logical.mean,logical.count
0,POSIXct,start_date,2010-01-01 01:15:00,2020-12-30 14:00:00,2015-09-03 10:23:00,111948.0,NaN,NaN,NaN,NaN,NaN
1,POSIXct,end_date,2010-01-03 10:00:00,2020-12-31 15:00:00,2015-09-12 08:22:30,69251.0,NaN,NaN,NaN,NaN,NaN
101,factor,discharge,NaT,NaT,NaT,NaN,0.0,8.0,"hom: 113207, dis: 3300, tra: 2983, dea: 1608",NaN,NaN
102,factor,gender,NaT,NaT,NaT,NaN,0.0,2.0,"W: 66098, M: 56338",NaN,NaN
103,factor,dementia,NaT,NaT,NaT,NaN,0.0,2.0,"Fal: 121761, Tru: 675",NaN,NaN
104,logical,delir,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.021603,"FAL: 119791, TRU: 2645"
105,logical,neuro_psy,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.071319,"FAL: 113704, TRU: 8732"
106,logical,any_medication,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.062441,"FAL: 114791, TRU: 7645"
107,logical,any_grouped_medication,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.062441,"FAL: 114791, TRU: 7645"
108,logical,any_acute_medication,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.052003,"FAL: 116069, TRU: 6367"


In [4]:
data.loc[data["skim_type"] == "character"].iloc[:, [0, 1, 8,9,10,11,12,13]]

,skim_type,skim_variable,character.min,character.max,character.empty,character.n_unique,character.whitespace,factor.ordered
2,character,id,36.0,36.0,0.0,122436.0,0.0,NaN
3,character,encounter_id,20.0,20.0,0.0,122436.0,0.0,NaN
4,character,patient_id,16.0,16.0,0.0,88166.0,0.0,NaN
5,character,admission_type,3.0,3.0,0.0,1.0,0.0,NaN
6,character,admission_reason,21.0,28.0,0.0,8.0,0.0,NaN
7,character,last_encounter,4.0,5.0,0.0,2.0,0.0,NaN
8,character,birth_date,19.0,19.0,0.0,96.0,0.0,NaN
9,character,chf,4.0,5.0,0.0,2.0,0.0,NaN
10,character,carit,4.0,5.0,0.0,2.0,0.0,NaN
11,character,valv,4.0,5.0,0.0,2.0,0.0,NaN


All character features with 1 or 2 unique attributes with minimum 4 characters and maximum 5 characters are likely boolean features (True/False)  
All features with more than 10.000 empty values can be imputed with NaN values, as they had in the original datasets almost only NaN values

In [5]:
data.loc[data["skim_type"] == "numeric"].iloc[:, [0, 1,18,19,20,21,22,23,24,25]]

,skim_type,skim_variable,numeric.mean,numeric.sd,numeric.p0,numeric.p25,numeric.p50,numeric.p75,numeric.p100,numeric.hist
109,numeric,reason_code,175.535888,135.536352,101.000000,101.000000,101.000000,201.000000,807.000000,▇▁▁▁▁
110,numeric,discharge_disposition,25.034214,18.786175,11.000000,19.000000,19.000000,19.000000,179.000000,▇▁▁▁▁
111,numeric,los,7.774290,10.029633,0.000000,3.000000,5.000000,9.000000,380.000000,▇▁▁▁▁
112,numeric,encounter_sequence,1.690067,1.462698,1.000000,1.000000,1.000000,2.000000,52.000000,▇▁▁▁▁
113,numeric,age_at_visit,52.881946,18.917783,18.000000,36.000000,54.000000,69.000000,107.000000,▆▆▇▅▁
114,numeric,eci_vw,3.451379,6.712049,-14.000000,0.000000,0.000000,4.000000,53.000000,▁▇▁▁▁
115,numeric,eci_swiss,4.443611,10.466000,-22.000000,0.000000,0.000000,9.000000,79.000000,▂▇▁▁▁
116,numeric,total_number_operations,2.905616,3.707745,1.000000,1.000000,2.000000,3.000000,103.000000,▇▁▁▁▁
117,numeric,total_opeartion_duration,333.479034,726.314298,0.000000,42.000000,113.000000,314.000000,21429.000000,▇▁▁▁▁
118,numeric,number_of_surgeries,2.905616,3.707745,1.000000,1.000000,2.000000,3.000000,103.000000,▇▁▁▁▁


In [6]:
pd.set_option('display.max_rows', 10)

In [7]:
adjusted_data = data.copy()

Create an empty dataframe. For every feature create one column with 100 rows

In [8]:
generated_df = pd.DataFrame(0, index=range(100), columns=adjusted_data["skim_variable"].values, dtype='object')

In [9]:
generated_df

,start_date,end_date,id,encounter_id,patient_id,admission_type,admission_reason,last_encounter,birth_date,chf,...,pipamperon_administered_acute,propofol_administered_acute,quetiapin_administered_acute,risperidon_administered_acute,cocaine,cannabis,barbiturate,opiate,amphetamin,tox_screen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Generate data for POSIXct, factor and logical features

In [10]:
adjusted_data.loc[adjusted_data["skim_type"].isin(["logical", "factor", "POSIXct"])].iloc[:, [0, 1, 4,5,6,7,13,14,15,16,17]]

,skim_type,skim_variable,POSIXct.min,POSIXct.max,POSIXct.median,POSIXct.n_unique,factor.ordered,factor.n_unique,factor.top_counts,logical.mean,logical.count
0,POSIXct,start_date,2010-01-01 01:15:00,2020-12-30 14:00:00,2015-09-03 10:23:00,111948.0,NaN,NaN,NaN,NaN,NaN
1,POSIXct,end_date,2010-01-03 10:00:00,2020-12-31 15:00:00,2015-09-12 08:22:30,69251.0,NaN,NaN,NaN,NaN,NaN
101,factor,discharge,NaT,NaT,NaT,NaN,0.0,8.0,"hom: 113207, dis: 3300, tra: 2983, dea: 1608",NaN,NaN
102,factor,gender,NaT,NaT,NaT,NaN,0.0,2.0,"W: 66098, M: 56338",NaN,NaN
103,factor,dementia,NaT,NaT,NaT,NaN,0.0,2.0,"Fal: 121761, Tru: 675",NaN,NaN
104,logical,delir,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.021603,"FAL: 119791, TRU: 2645"
105,logical,neuro_psy,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.071319,"FAL: 113704, TRU: 8732"
106,logical,any_medication,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.062441,"FAL: 114791, TRU: 7645"
107,logical,any_grouped_medication,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.062441,"FAL: 114791, TRU: 7645"
108,logical,any_acute_medication,NaT,NaT,NaT,NaN,NaN,NaN,NaN,0.052003,"FAL: 116069, TRU: 6367"


In [11]:
np.random.seed(42) 

#create datetime data with uniform distribution
for i in range(0,2):
    min_date = adjusted_data.iloc[i,4]
    median_date = adjusted_data.iloc[i,6]
    max_date = adjusted_data.iloc[i,5]
    dates_before_median = np.linspace(min_date.value, median_date.value, 50, endpoint=False)
    dates_after_median = np.linspace(median_date.value, max_date.value, 50, endpoint=True)
    all_dates = np.concatenate([dates_before_median, dates_after_median])
    generated_df.iloc[:,i] = pd.to_datetime(all_dates)

# generate the factor data based on the counts
# do the same for the logical attributed
for i in range(101,109):
    if i < 104:
        cols=15
    else:
        cols=17
    data_dict = {item.split(': ')[0]: int(item.split(': ')[1]) for item in adjusted_data.iloc[i,cols].split(",")}
    total_count = sum(data_dict.values())
    vals = []
    for j in data_dict:
        vals += [j] * np.ceil(100 * data_dict[j] / total_count).astype(int)     #ceil, to ensure generating at least 100 values
    generated_df.iloc[:,i] = np.random.choice(vals, 100)  # Randomly select 100 values from the generated list    

C:\Users\Luca\AppData\Local\Temp\ipykernel_10272\688359677.py:11: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  generated_df.iloc[:,i] = pd.to_datetime(all_dates)
C:\Users\Luca\AppData\Local\Temp\ipykernel_10272\688359677.py:11: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  generated_df.iloc[:,i] = pd.to_datetime(all_dates)


In [12]:
generated_df.iloc[:,[0,1,101,102,103,104,105,106,107,108]]

,start_date,end_date,discharge,gender,dementia,delir,neuro_psy,any_medication,any_grouped_medication,any_acute_medication
0,2010-01-01 01:15:00.000000000,2010-01-03 10:00:00.000000000,hom,W,Fal,FAL,FAL,FAL,FAL,FAL
1,2010-02-11 11:30:45.600000000,2010-02-13 23:24:27.000000000,hom,W,Fal,FAL,FAL,FAL,FAL,FAL
2,2010-03-24 21:46:31.200000000,2010-03-27 12:48:54.000000000,hom,M,Fal,TRU,FAL,FAL,FAL,FAL
3,2010-05-05 08:02:16.800000000,2010-05-08 02:13:21.000000000,hom,M,Fal,FAL,FAL,FAL,TRU,FAL
4,2010-06-15 18:18:02.400000000,2010-06-18 15:37:48.000000000,hom,W,Fal,FAL,FAL,FAL,FAL,FAL
...,...,...,...,...,...,...,...,...,...,...
95,2020-07-24 19:05:33.061224448,2020-07-26 11:31:13.469387776,hom,W,Fal,FAL,TRU,FAL,FAL,FAL
96,2020-09-02 11:49:09.795918336,2020-09-04 00:23:25.102040832,hom,W,Fal,FAL,FAL,FAL,FAL,FAL
97,2020-10-12 04:32:46.530612224,2020-10-13 13:15:36.734693888,hom,M,Fal,FAL,FAL,FAL,FAL,FAL
98,2020-11-20 21:16:23.265306112,2020-11-22 02:07:48.367346944,hom,M,Fal,FAL,FAL,FAL,FAL,FAL


Generate data for character features

In [13]:
adjusted_data.loc[adjusted_data["skim_type"] == "character"].iloc[:, [0, 1, 8,9,10,11,12]]

,skim_type,skim_variable,character.min,character.max,character.empty,character.n_unique,character.whitespace
2,character,id,36.0,36.0,0.0,122436.0,0.0
3,character,encounter_id,20.0,20.0,0.0,122436.0,0.0
4,character,patient_id,16.0,16.0,0.0,88166.0,0.0
5,character,admission_type,3.0,3.0,0.0,1.0,0.0
6,character,admission_reason,21.0,28.0,0.0,8.0,0.0
...,...,...,...,...,...,...,...
96,character,quetiapin_first_administration_date,0.0,25.0,121534.0,889.0,0.0
97,character,risperidon_clinic,0.0,6.0,122057.0,22.0,0.0
98,character,risperidon_department,0.0,3.0,122057.0,10.0,0.0
99,character,risperidon_first_administration_date,0.0,25.0,122057.0,378.0,0.0


In [14]:
adjusted_data.loc[(adjusted_data["skim_type"] == "character") & (((adjusted_data["character.min"] != 4) | 
                                                                  (adjusted_data["character.min"] != 5)) & (adjusted_data["character.max"] != 5) ) 
                  & (adjusted_data["character.empty"] < 100000)].iloc[:, [0, 1, 8,9,10,11,12]]

,skim_type,skim_variable,character.min,character.max,character.empty,character.n_unique,character.whitespace
2,character,id,36.0,36.0,0.0,122436.0,0.0
3,character,encounter_id,20.0,20.0,0.0,122436.0,0.0
4,character,patient_id,16.0,16.0,0.0,88166.0,0.0
5,character,admission_type,3.0,3.0,0.0,1.0,0.0
6,character,admission_reason,21.0,28.0,0.0,8.0,0.0
8,character,birth_date,19.0,19.0,0.0,96.0,0.0
59,character,first_operation_code,5.0,8.0,0.0,5636.0,0.0
60,character,major_surgery,4.0,31.0,0.0,18.0,0.0


In [15]:
def generate_random_string(length):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

for i in [2,3,4]: #generate random strings for id columns
    val = []
    for j in range(100):
        val.append(generate_random_string(int(adjusted_data.iloc[i,9])))
    generated_df.iloc[:,i] = val

#admission_type
i = 5
generated_df.iloc[:,i] = ["001" for _ in range(100)] #fill with one unique value

#admisson_reason
i = 6
admission_reasons = [   #eight admission reasons from gpt
    "Severe abdominal pain",
    "Chest pain and pressure",
    "High fever with chills",
    "Shortness of breath",
    "Acute allergic reaction",
    "Loss of consciousness",
    "Severe dehydration",
    "Persistent severe headache"
]
random_reasons = [random.choice(admission_reasons) for _ in range(100)]
generated_df.iloc[:,i] = random_reasons

#birth_date
i = 8
start_date = datetime(1940, 1, 1)
end_date = datetime(2005, 1, 1)
date_range = (end_date - start_date).days
birthdays = [start_date + timedelta(days=random.randint(0, date_range)) for _ in range(100)]
generated_df.iloc[:,i] = birthdays

#first_operation_code
i=59
val = []
for j in range(100):
    random_length = random.randint(int(adjusted_data.iloc[i,8]), int(adjusted_data.iloc[i,9]))
    val.append(generate_random_string(random_length))
generated_df.iloc[:,i] = val
    
#major_surgery
i=60
major_surgeries = [     #generated with gpt
    "Appendectomy","Cholecystectomy", "Coronary bypass","Lung transplant", "Hip replacement", "Knee replacement","Spinal fusion","Heart valve repair", "Liver transplant", 
    "Cataract surgery","Gallbladder removal", "Kidney transplant","Mastectomy","Hernia repair", "Thyroidectomy","Hysterectomy","Arthroscopy","Brain tumor removal"  
]
random_surgeries = [random.choice(major_surgeries) for _ in range(100)]
generated_df.iloc[:,i] = random_surgeries


# columns with False/True character values
idxs = adjusted_data.loc[(adjusted_data["skim_type"] == "character") & (adjusted_data["character.min"] == 4) & (adjusted_data["character.max"] == 5)].index
for i in idxs:
    vals = ["True"] * np.ceil(random.randint(0,100)/1000 * 100).astype(int)   # create a random number between 0 and 0.1 for the rate of True values
    vals = vals + ["False"] * (100 - len(vals))
    generated_df.iloc[:,i] = np.random.choice(vals, 100) #shuffle the data
    
#columns with only false values
idxs = adjusted_data.loc[(adjusted_data["skim_type"] == "character") & (adjusted_data["character.min"] == 5) & (adjusted_data["character.max"] == 5)].index
for i in idxs:
    generated_df.iloc[:,i] = ["False"] * 100
    
#fill columns with nothing that have more that 100 000 character.empty values
idxs = adjusted_data.loc[(adjusted_data["skim_type"] == "character") & (adjusted_data["character.empty"] >= 100000)].index
for i in idxs:
    generated_df.iloc[:,i] = [""] * 100

C:\Users\Luca\AppData\Local\Temp\ipykernel_10272\1846096300.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  generated_df.iloc[:,i] = birthdays


Generate data for numeric features   
Only create integer values as the quantiles indicate that every feature only contained integer values

In [16]:
adjusted_data.loc[adjusted_data["skim_type"] == "numeric"].iloc[:, [0, 1,18,19,20,21,22,23,24,25]]

,skim_type,skim_variable,numeric.mean,numeric.sd,numeric.p0,numeric.p25,numeric.p50,numeric.p75,numeric.p100,numeric.hist
109,numeric,reason_code,175.535888,135.536352,101.0,101.0,101.0,201.0,807.0,▇▁▁▁▁
110,numeric,discharge_disposition,25.034214,18.786175,11.0,19.0,19.0,19.0,179.0,▇▁▁▁▁
111,numeric,los,7.774290,10.029633,0.0,3.0,5.0,9.0,380.0,▇▁▁▁▁
112,numeric,encounter_sequence,1.690067,1.462698,1.0,1.0,1.0,2.0,52.0,▇▁▁▁▁
113,numeric,age_at_visit,52.881946,18.917783,18.0,36.0,54.0,69.0,107.0,▆▆▇▅▁
...,...,...,...,...,...,...,...,...,...,...
408,numeric,cannabis,0.000776,0.027845,0.0,0.0,0.0,0.0,1.0,▇▁▁▁▁
409,numeric,barbiturate,0.000768,0.027698,0.0,0.0,0.0,0.0,1.0,▇▁▁▁▁
410,numeric,opiate,0.000776,0.027845,0.0,0.0,0.0,0.0,1.0,▇▁▁▁▁
411,numeric,amphetamin,0.000800,0.028280,0.0,0.0,0.0,0.0,1.0,▇▁▁▁▁


In [17]:
for i in adjusted_data.loc[adjusted_data["skim_type"] == "numeric"].index:
    mean = adjusted_data.iloc[i,18]
    std = adjusted_data.iloc[i,19]
    p0 = adjusted_data.iloc[i,20]
    p25 = adjusted_data.iloc[i,21]
    p50 = adjusted_data.iloc[i,22]
    p75 = adjusted_data.iloc[i,23]
    p100 = adjusted_data.iloc[i,24]

In [18]:
for i in adjusted_data.loc[adjusted_data["skim_type"] == "numeric"].index:
    mean = adjusted_data.iloc[i,18]
    std = adjusted_data.iloc[i,19]
    p0 = adjusted_data.iloc[i,20]
    p25 = adjusted_data.iloc[i,21]
    p50 = adjusted_data.iloc[i,22]
    p75 = adjusted_data.iloc[i,23]
    p100 = adjusted_data.iloc[i,24]

    values = np.random.normal(loc=mean, scale=std, size=100)
    # Clip values to ensure they are within the range [p0, p100]
    values = np.clip(values, p0, p100)
    values = np.round(values).astype(int)
    # Ensure the distribution reflects percentiles
    values.sort()
    values[:25] = np.clip(values[:25], p0, p25)
    values[25:75] = np.clip(values[25:75], p25, p75)
    values[75:] = np.clip(values[75:], p75, p100)
    random.shuffle(values)
    generated_df.iloc[:,i] = values

#overwrite columns with nothing that have more that 100 000 numeric.empty values
for i in adjusted_data.loc[adjusted_data["n_missing"] >= 100000].index:
    generated_df.iloc[:,i] = [np.nan] * 100

C:\Users\Luca\AppData\Local\Temp\ipykernel_10272\2842605521.py:20: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  generated_df.iloc[:,i] = values


In [19]:
#save the dataset
generated_df.to_csv("generated_data.csv", index=False)

Generate the Labels

In [ ]:
def print_column(i):
    print(i)
    print(generated_df[i].unique()[:10])

#label the dataset manually
labels=[]
for i in generated_df.columns:
    inp = "a"
    while inp == "a":
        print_column(i)
        inp = input("Enter the label for the column: ")
    if inp == "p":
        labels.append("personal")
    elif inp == "n":
        labels.append("non-personal")
    else:
        labels.append("dropa")
    print()

In [39]:
test_labels = pd.DataFrame(labels).rename(columns={0:"label"})
test_labels.to_csv("generated_data_labels.csv", index=False)

In [21]:
test_labels.value_counts()

label   
drop        381
personal     32
dtype: int64

In [22]:
pd.set_option('display.max_rows', 500)
test_labels.index = generated_df.columns
test_labels.loc[test_labels["label"] != "drop"]

,label
encounter_id,personal
patient_id,personal
admission_type,personal
admission_reason,personal
last_encounter,personal
birth_date,personal
aids,personal
wloss,personal
alcohol,personal
drug,personal


In [23]:
pd.set_option('display.max_rows', 10)
test_labels.loc[test_labels["label"] != "drop"].to_csv("test_labels_personal.csv", index=False)
generated_df.loc[:,test_labels.loc[test_labels["label"] != "drop"].index].to_csv("test.csv", index=False)

Check data of Mr. Tsialis  
Data of Mr. Tsialis does not include all columns, which I have picked for my experiments

In [47]:
test_df = pd.read_csv("test.csv")
data_tsialis = pd.read_csv("gen_data_tsialis.csv")
data_tsialis[list( set(data_tsialis.columns).intersection(set(test_df.columns)) )].head()

,depre,psycho,gender,ischemic_stroke,dementia,aids,birth_date,admission_reason,drug,discharge,first_operation_code,hemorhage_stroke,patient_id,encounter_id,admission_type,last_encounter,wloss,age_at_visit,total_number_operations,alcohol
0,True,False,M,True,False,False,1988-01-01 00:01:00,Normal Child delivery,True,discharge to psychiatric therapy,5-749.0,True,F86020D1ED22872A,F86020D1E52F872F-005,IMP,True,True,-8.000000,1.0,True
1,True,True,X,True,True,True,1935-01-01 00:01:00,Emergency Hospital treatment,True,discharge to hospitz,NaN,False,F86020D0E82F862E,F86020D1E82E822E-001,IMP,True,True,-4.162662,1.0,False
2,True,False,X,False,False,True,2000-01-01 00:01:00,Emergency Organ procurement,True,death,5-576.60,True,F86020D2E427872E,F86020D1E82E822E-001,IMP,False,False,-0.291226,NaN,True
3,True,False,U,False,False,False,1965-01-01 00:01:00,Work Hospital treatment,False,discharge to psychiatric therapy,NaN,False,F86020D1E8228329,F86020D0ED21802E-002,IMP,True,True,4.976455,1.0,True
4,True,False,X,False,False,True,1990-01-01 00:01:00,Work Hospital treatment,True,discharge to psychiatric therapy,5-449.e3,True,F86020D0E5268B2B,F86020D1E823832B-001,IMP,False,False,5.012137,1.0,False


Use the data from Mr. Tsialis  
For the columns which are not present in Mr. Tsialis dataset, use your own data

In [51]:
for i in test_df.columns:
    if i in data_tsialis.columns:
        test_df[i] = data_tsialis[i]
test_df.to_csv("test.csv", index=False)